In [1]:
import re 
import sys
import yaml 
import shutil
import pandas as pd
from collections import defaultdict
import logging
import os



In [2]:
import datetime 
now = datetime.datetime.now()
start_time = now.strftime("%Y-%m-%d %H:%M:%S")
import os
os.environ['start_time'] = start_time

In [3]:
from datetime import datetime

class RunTime:
    def __init__(self, start_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")):
        self.start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

    def elapsed(self):
        self.end_time = datetime.now()
        return f'Duration: {self.end_time - self.start_time}'

In [4]:
# cols
RED = "\033[;31;1m"
RED_ = "\033[;31;4m"
GRE = "\033[;32;1m"
YEL = "\033[;33;1m"
BLU = "\033[;34;1m"
PRP = "\033[;35;1m"
CYN = "\033[;36;1m"
BLD = "\033[;37;1m"
GRY = "\033[;30;1m"
NC = "\033[;39;0m"
NC_ = "\033[;39;4m"

start_time = os.environ['start_time']
runtime = RunTime(start_time)



# other vars
global_vars = {}




In [5]:
class CustomFormatter(logging.Formatter):
    ### code from:
    ## https://stackoverflow.com/questions/384076/how-can-i-color-python-logging-output

    format = "[%(asctime)s] %(levelname)s: %(message)s"
    datefmtstr = "%Y-%m-%d %H:%M:%S"

    FORMATS = {
        logging.DEBUG: format.replace('%(levelname)s', GRY + '%(levelname)s' + NC),
        logging.INFO: format.replace('%(levelname)s', GRY + '%(levelname)s' + NC),
        logging.WARNING: format.replace('%(levelname)s', YEL + '%(levelname)s' + NC),
        logging.ERROR: format.replace('%(levelname)s', RED + '%(levelname)s' + NC),
        logging.CRITICAL: format.replace('%(levelname)s', RED_ + '%(levelname)s' + NC),
    }

    def format(self, record):
        log_fmt = self.FORMATS.get(record.levelno)
        formatter = logging.Formatter(log_fmt, datefmt=self.datefmtstr)
        return formatter.format(record)
    

class GLogger:
    def __init__(self):
        # create logger attr and set level
        self.logger = logging.getLogger('GUAP_logger')
        self.formatter = logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

    def create_console_handler(self, verbose=False):
        # create handler
        self.ch = logging.StreamHandler()

        # set level according to user verbose option
        if verbose:
            self.ch.setLevel(logging.DEBUG)
        else:
            self.ch.setLevel(logging.WARNING)

        self.ch.setFormatter(CustomFormatter())

        # add handler
        self.logger.addHandler(self.ch)

    def create_file_handler(self, file):
        # create handler
        self.fh = logging.FileHandler(file)
        self.fh.setLevel(logging.DEBUG)
        self.fh.setFormatter(self.formatter)

        self.logger.addHandler(self.fh)


    def prnt_info(self, str):
        self.logger.info(f"{str}")


    def prnt_warning(self, str):
        self.logger.warning(f"{str}")


    def prnt_error(self, str):
        self.logger.error(f"{str}")


    def prnt_fatel(self, str):
        self.logger.fatal(f"{str}")
        print(f"{PRP}{runtime.elapsed()}{NC}")
        exit(1)



In [6]:
glogger = GLogger()

In [7]:

def check_extension(df): # takes pandas df and returns string
    # checks all files have same extension from pandas df, to use in generete sample table function
    uniques = df['ext'].unique()
    if len(uniques) > 1:
        glogger.prnt_fatel(f"Your input directory has multible fastq file extensions, please check directory.")

    else:
        return uniques[0]


def check_PE(df): # takes pandas df and returns string
    """checks all files either single or paried ended from pandas df, to use in generete sample table function"""
    uniques = df['PE'].unique()
    if len(uniques) > 1:
        glogger.prnt_fatel(f"Your input directory has both Paired and single files, please check directory.")
    else:
        return uniques[0]


def check_R(df): # takes pandas df and returns string
    """checks all files have same naming patterns from pandas df, to use in generete sample table function"""
    uniques = df['read_num'].unique()
    if len(uniques) > 1:
        glogger.prnt_fatel(f"Your input directory has multible fastq file naming patterns, please check directory.")
    else:
        return uniques[0].replace("1","")


def check_pattern(df): # takes pandas df and returns a string 
    """checks all files have same naming patterns from pandas df, to use in generete sample table function"""
    uniques = df['matched_pattern'].unique()
    if len(uniques) > 1:
        glogger.prnt_fatel(f"Your input directory has multible fastq file naming patterns, please check directory.")
    else:
        return uniques[0]
    

In [10]:
def recogize_pattern(file_name): # takes string of fastq file name and returns dict with read info and id
    """ using re to recognize the naming pattern of samples (illumina, srr and general naming patten)"""
    # naming pattern for re 
    patterns = { # ! fix (_|\.) group for R pattern in dict config !
        "Novagen": "(((.+)-(.+))([1|2])\.(fastq\.gz|fastq|fq\.gz|fq))",
        "illumina": "(((.+)_(S\d+)_(L00\d))_(R1|R2|r1|r2|read1|read2)_(00\d)\.(fastq\.gz|fastq|fq\.gz|fq))",
        "SRR": "(((SRR)(\d+))(_|\.)(1|2|R1|R2|r1|r2|read1|read2)\.(fastq\.gz|fastq|fq\.gz|fq))",
        "general": "(((.+))(_|\.)(1|2|R1|R2|r1|r2|read1|read2)\.(fastq\.gz|fastq|fq\.gz|fq))"
    }
## 1: full name 2: without read group and ext, 3: sample name, 4: flowcell + lane, 5: read number, 6: ext
    matched_pattern = None
    ## loop on pattern to and checks whichs one matches 
    ## starting with illumina because general would match any ways
    ## breaks once successful 
    for ptrn_name, pattern in patterns.items():
        try:
            matched = re.match(pattern, file_name) 
        except:
            continue
        
        if bool(matched) :
            matched_pattern = ptrn_name
            break
            
        else:
            continue
## 1: full name 2: without read group and ext, 3: sample name, 4: flowcell + lane, 5: read number, 6: ext

    if matched_pattern == "Novagen":
        sample, sample_name, sample_id, unit, read_num, ext = matched.groups()[0], matched.groups()[1], matched.groups()[2], matched.groups()[3], matched.groups()[4], matched.groups()[5]
        # sample, sample_name, sample_id, unit, library_index, acc1, acc2, lane, R_pattern, R_sep, read_num, ext, tail, sample_number =  matched.groups()[0], matched.groups()[1], matched.groups()[2], matched.groups()[2+1], matched.groups()[3+1], matched.groups()[4+1], matched.groups()[5+1], matched.groups()[6+1], matched.groups()[7+1] , matched.groups()[8+1] , matched.groups()[9+1], matched.groups()[10+1],"", ""

    # elif matched_pattern == "Novagen2":
    #     file_name, sample_name, sample_id, unit, library_index, acc1, acc2, lane, R_pattern, R_sep, read_num, ext, tail, sample_number =  matched.groups()[0], matched.groups()[1], matched.groups()[2], matched.groups()[2+1], matched.groups()[3+1], matched.groups()[4+1], matched.groups()[5+1], matched.groups()[6+1], matched.groups()[7+1] , matched.groups()[8+1] , matched.groups()[9+1], matched.groups()[10+1],"", ""
    # elif matched_pattern == "illumina":
    #     file_name, sample_name, sample_id, sample_number, read_num, lane, tail, ext = matched.groups()[0], matched.groups()[1], matched.groups()[2], matched.groups()[3], matched.groups()[5], matched.groups()[4], matched.groups()[6], matched.groups()[7]

    # elif matched_pattern == "SRR":
    #     glogger.prnt_fatel(f"{RED}{matched_pattern}{NC} is currntly not supported sample naming pattern\nonly {GRE}'Illumina'{NC} naming pattern is supported at the moment")
    #     file_name, sample_name, sample_id, sample_number, read_num, lane, tail, ext = matched.groups()[0], matched.groups()[1], matched.groups()[3], "", matched.groups()[5], "", "", matched.groups()[6]

    # elif matched_pattern == "general":
    #     glogger.prnt_fatel(f"{RED}{matched_pattern}{NC} is currntly not supported sample naming pattern\nonly {GRE}'Illumina'{NC} naming pattern is supported at the moment")
    #     file_name, sample_name, sample_id, sample_number, read_num, lane, tail, ext = matched.groups()[0], matched.groups()[1], matched.groups()[1], "", matched.groups()[4], "", "", matched.groups()[5]

    else:
        glogger.prnt_fatel(f"{RED}Your Samples Pattern is an unfamiler pattern.{NC}\nPlease contact my Developpers and they will look into it :D")
        file_name = sample_name = sample_id = sample_number = read_num = lane = tail = ext = None


    if matched_pattern == "Novagen":
        return {
            "file_name": sample,
            "unit": unit,
            "sample_name": sample_name,
            "sample_id": sample_id,
            "read_num" : read_num, 
            "ext" : ext,
            "matched_pattern": ptrn_name
        }
    
        # return {
        #     "file_name": file_name,
        #     "unit": unit,
        #     "sample_name": sample_name,
        #     "sample_id": sample_id,
        #     "library_index": library_index,
        #     "acc1": acc1,
        #     "acc2": acc2,
        #     "lane": lane,
        #     "R_pattern" : R_pattern, 
        #     "R_sep" : R_sep, 
        #     "read_num" : read_num, 
        #     "ext" : ext,
        #     "tail": tail,
        #     "sample_number": sample_number,
        #     "matched_pattern": ptrn_name
        # }
    
    else:
        # Returns a dictionary of sample information
        return {
            "file_name": file_name,
            "sample_name": sample_name,
            "sample_id": sample_id,
            "sample_number": sample_number,
            "read_num": read_num,
            "lane": lane,
            "tail": tail,
            "ext": ext,
            "matched_pattern": ptrn_name
        }


In [11]:
def parse_samples(inpath): # takes path return contains fastq files, returns df contains sample information
    ## takes input path
    ## gets the file names containg fastq and fq
    ## performs the recogize_pattern function to 
    ## capture sample information and stores it in 
    ## pandas df
    # input path to absolute path
    path = os.path.abspath(inpath)
    # list all files
    all_files = os.listdir(path)
    samples = defaultdict(dict)
    # takes fastq files only
    for file_name in all_files:
        if os.path.isfile(path + "/" + file_name) and ("fastq" in file_name or "fq" in file_name):
            # Captures the file path and name
            filename, file_extension = os.path.splitext(file_name)
            if "fastq" in filename or "fq" in filename:
                filename, new_ext = os.path.splitext(filename)
                file_extension = new_ext + file_extension
            # recogize_pattern function returns a dictitionary with sample names, id, and read information
            sample_info = recogize_pattern(file_name)

            # get only forward reads and replace the read number to get R2
            # appends sample information to a dict of dicts
            
            if "1" in sample_info["read_num"]:
                read_2 = sample_info["read_num"].replace("1","2")
                if sample_info["matched_pattern"] == "illumina":
                    read_1 = f"{sample_info['read_num']}_{sample_info['tail']}.f"
                    read_2 = f"{read_2}_{sample_info['tail']}.f"
                else:
                    read_1 = f"{sample_info['read_num']}.f"
                    read_2 = f"{read_2}.f"

                f2 = file_name.replace(read_1, read_2)
                if f2 in all_files:
                    sample_info["file2"] = f2
                    sample_info["PE"] = True
                    samples[sample_info["sample_name"]] = sample_info

                else:
                    sample_info["file2"] = ""
                    sample_info["PE"] = False

    # converts the dict to pandas df and returns the df
    m_samples = samples
    samples= pd.DataFrame(samples).T
    samples = samples.sort_values(by=['sample_id'])

    return samples

In [12]:
samples = parse_samples("empty_sample_names")

In [13]:
samples

,file_name,unit,sample_name,sample_id,read_num,ext,matched_pattern,file2,PE
RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_1.fq.gz,1A_HT3H7DSX5_L2_,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_,RZ100M_CKDN22H000023,1,fq.gz,Novagen,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_2.fq.gz,True
RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_1.fq.gz,1A_HT3F3DSX5_L1_,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_,RZ100M_CKDN22H000023,1,fq.gz,Novagen,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_2.fq.gz,True
RZ100_CKDN220028927-1A_H3FYWDSX5_L1_,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_1.fq.gz,1A_H3FYWDSX5_L1_,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_,RZ100_CKDN220028927,1,fq.gz,Novagen,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_2.fq.gz,True
RZ100_CKDN220028927-1A_H3G37DSX5_L3_,RZ100_CKDN220028927-1A_H3G37DSX5_L3_1.fq.gz,1A_H3G37DSX5_L3_,RZ100_CKDN220028927-1A_H3G37DSX5_L3_,RZ100_CKDN220028927,1,fq.gz,Novagen,RZ100_CKDN220028927-1A_H3G37DSX5_L3_2.fq.gz,True
RZ101M_CKDN220028930-1A_H3GCHDSX5_L1_,RZ101M_CKDN220028930-1A_H3GCHDSX5_L1_1.fq.gz,1A_H3GCHDSX5_L1_,RZ101M_CKDN220028930-1A_H3GCHDSX5_L1_,RZ101M_CKDN220028930,1,fq.gz,Novagen,RZ101M_CKDN220028930-1A_H3GCHDSX5_L1_2.fq.gz,True
...,...,...,...,...,...,...,...,...,...
S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_1.fq.gz,1A_HFW5VDSX3_L1_,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_,S_ZIKA_23B_CKDN210020708,1,fq.gz,Novagen,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_2.fq.gz,True
S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_1.f...,1A_HFW5VDSX3_L2_,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_,S_ZIKA_23Mom_CKDN210020709,1,fq.gz,Novagen,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_2.f...,True
S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_1.f...,1A_HFW5VDSX3_L3_,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_,S_ZIKA_24Mom_CKDN210020710,1,fq.gz,Novagen,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_2.f...,True
S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_1....,1A_HFW5VDSX3_L2_,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_,S_ZIKA_25Baby_CKDN210020712,1,fq.gz,Novagen,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_2....,True


In [12]:
samples.to_csv("samples.tsv",sep='\t',index=False)

In [13]:
sample_table_file=('samples.tsv')

SampleTable = pd.read_table(sample_table_file)


In [14]:
SampleTable

,file_name,unit,sample_name,sample_id,library_index,acc1,acc2,lane,R_pattern,R_sep,read_num,ext,tail,sample_number,matched_pattern,file2,PE
0,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_1.fq.gz,CKDN220028927-1A_H3FYWDSX5_L1,RZ100_CKDN220028927-1A_H3FYWDSX5_L1,RZ100,CKDN220028927-1A_H3FYWDSX5,CKDN220028927,1A_H3FYWDSX5,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_2.fq.gz,True
1,RZ100_CKDN220028927-1A_H3G37DSX5_L3_1.fq.gz,CKDN220028927-1A_H3G37DSX5_L3,RZ100_CKDN220028927-1A_H3G37DSX5_L3,RZ100,CKDN220028927-1A_H3G37DSX5,CKDN220028927,1A_H3G37DSX5,L3,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100_CKDN220028927-1A_H3G37DSX5_L3_2.fq.gz,True
2,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_1.fq.gz,CKDN22H000023-1A_HT3H7DSX5_L2,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2,RZ100M,CKDN22H000023-1A_HT3H7DSX5,CKDN22H000023,1A_HT3H7DSX5,L2,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_2.fq.gz,True
3,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_1.fq.gz,CKDN22H000023-1A_HT3F3DSX5_L1,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1,RZ100M,CKDN22H000023-1A_HT3F3DSX5,CKDN22H000023,1A_HT3F3DSX5,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_2.fq.gz,True
4,RZ101_CKDN220028929-1A_H3FYWDSX5_L1_1.fq.gz,CKDN220028929-1A_H3FYWDSX5_L1,RZ101_CKDN220028929-1A_H3FYWDSX5_L1,RZ101,CKDN220028929-1A_H3FYWDSX5,CKDN220028929,1A_H3FYWDSX5,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ101_CKDN220028929-1A_H3FYWDSX5_L1_2.fq.gz,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_1.fq.gz,CKDN210020708-1A_HFW5VDSX3_L1,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1,S_ZIKA_23B,CKDN210020708-1A_HFW5VDSX3,CKDN210020708,1A_HFW5VDSX3,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_2.fq.gz,True
1189,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_1.f...,CKDN210020709-1A_HFW5VDSX3_L2,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2,S_ZIKA_23Mom,CKDN210020709-1A_HFW5VDSX3,CKDN210020709,1A_HFW5VDSX3,L2,_1,_,1,fq.gz,NaN,NaN,Novagen1,S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_2.f...,True
1190,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_1.f...,CKDN210020710-1A_HFW5VDSX3_L3,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3,S_ZIKA_24Mom,CKDN210020710-1A_HFW5VDSX3,CKDN210020710,1A_HFW5VDSX3,L3,_1,_,1,fq.gz,NaN,NaN,Novagen1,S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_2.f...,True
1191,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_1....,CKDN210020712-1A_HFW5VDSX3_L2,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2,S_ZIKA_25Baby,CKDN210020712-1A_HFW5VDSX3,CKDN210020712,1A_HFW5VDSX3,L2,_1,_,1,fq.gz,NaN,NaN,Novagen1,S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_2....,True


In [15]:

files_R1s = list(SampleTable.iloc[:, 0])
files_R2s = list(SampleTable.iloc[:, 15])
samples = list(SampleTable.iloc[:, 2]) # sample full name
units = list(SampleTable.iloc[:, 1])
samples_IDs = list(SampleTable.iloc[:, 3])
library_index = list(SampleTable.iloc[:, 4])

In [16]:
samples

['RZ100_CKDN220028927-1A_H3FYWDSX5_L1',
 'RZ100_CKDN220028927-1A_H3G37DSX5_L3',
 'RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2',
 'RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1',
 'RZ101_CKDN220028929-1A_H3FYWDSX5_L1',
 'RZ101_CKDN220028929-1A_H3GCHDSX5_L3',
 'RZ101M_CKDN220028930-1A_H3FYWDSX5_L1',
 'RZ101M_CKDN220028930-1A_H3GCHDSX5_L1',
 'RZ102_CKDN220028931-1A_H3GCHDSX5_L3',
 'RZ102_CKDN220028931-1A_H3G2WDSX5_L2',
 'RZ102M_CKDN220028932-1A_H3GTLDSX5_L2',
 'RZ102M_CKDN220028932-1A_H3GCHDSX5_L3',
 'RZ103_CKDN220028933-1A_H3GCHDSX5_L3',
 'RZ103_CKDN220028933-1A_H3FYWDSX5_L1',
 'RZ103M_CKDN22H000024-1A_HT3H7DSX5_L2',
 'RZ103M_CKDN22H000024-1A_HT3F3DSX5_L1',
 'RZ104_CKDN220028935-1A_H3GCHDSX5_L1',
 'RZ104M_CKDN220028936-1A_H3GCHDSX5_L3',
 'RZ104M_CKDN220028936-1A_H3FYWDSX5_L1',
 'RZ105_CKDN220028937-1A_H3GCHDSX5_L3',
 'RZ105_CKDN220028937-1A_H3GTLDSX5_L2',
 'RZ105M_CKDN220028938-1A_H3GCHDSX5_L3',
 'RZ106_CKDN220028939-1A_H3FYWDSX5_L1',
 'RZ106_CKDN220028939-1A_H3GCHDSX5_L3',
 'RZ106M_CKDN220028940-1A_H3G

In [17]:
samples_IDs

['RZ100',
 'RZ100',
 'RZ100M',
 'RZ100M',
 'RZ101',
 'RZ101',
 'RZ101M',
 'RZ101M',
 'RZ102',
 'RZ102',
 'RZ102M',
 'RZ102M',
 'RZ103',
 'RZ103',
 'RZ103M',
 'RZ103M',
 'RZ104',
 'RZ104M',
 'RZ104M',
 'RZ105',
 'RZ105',
 'RZ105M',
 'RZ106',
 'RZ106',
 'RZ106M',
 'RZ106M',
 'RZ107',
 'RZ107M',
 'RZ107M',
 'RZ108',
 'RZ108M',
 'RZ108M',
 'RZ109',
 'RZ109',
 'RZ109M',
 'RZ109M',
 'RZ110',
 'RZ110',
 'RZ110M',
 'RZ110M',
 'RZ111',
 'RZ111',
 'RZ111M',
 'RZ111M',
 'RZ112',
 'RZ112M',
 'RZ113',
 'RZ113',
 'RZ113M',
 'RZ113M',
 'RZ114',
 'RZ114',
 'RZ114M',
 'RZ114M',
 'RZ115',
 'RZ115M',
 'RZ115M',
 'RZ116',
 'RZ116',
 'RZ117',
 'RZ118',
 'RZ118',
 'RZ118M',
 'RZ119',
 'RZ119',
 'RZ119M',
 'RZ120',
 'RZ120M',
 'RZ120M',
 'RZ121',
 'RZ121M',
 'RZ122',
 'RZ122M',
 'RZ122M',
 'RZ123',
 'RZ123M',
 'RZ124',
 'RZ124',
 'RZ124M',
 'RZ125',
 'RZ125M',
 'RZ126',
 'RZ126M',
 'RZ126M',
 'RZ127',
 'RZ127M',
 'RZ128',
 'RZ128',
 'RZ128M',
 'RZ128M',
 'RZ129',
 'RZ129',
 'RZ129M',
 'RZ130',
 'RZ130M',
 'R

In [18]:

units = (
    pd.read_csv('samples.tsv', sep="\t", dtype={"sample_id": str, "library_index": str, "lane": str})
    .set_index(["sample_id", "unit"], drop=False)
    .sort_index()
)


In [19]:
units.to_csv("units.tsv",sep='\t',index=False)

In [20]:
units

file_name  \
sample_id     unit                                                                               
RZ100         CKDN220028927-1A_H3FYWDSX5_L1        RZ100_CKDN220028927-1A_H3FYWDSX5_L1_1.fq.gz   
              CKDN220028927-1A_H3G37DSX5_L3        RZ100_CKDN220028927-1A_H3G37DSX5_L3_1.fq.gz   
RZ100M        CKDN22H000023-1A_HT3F3DSX5_L1       RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1_1.fq.gz   
              CKDN22H000023-1A_HT3H7DSX5_L2       RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2_1.fq.gz   
RZ101         CKDN220028929-1A_H3FYWDSX5_L1        RZ101_CKDN220028929-1A_H3FYWDSX5_L1_1.fq.gz   
...                                                                                        ...   
S_ZIKA_23B    CKDN210020708-1A_HFW5VDSX3_L1   S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1_1.fq.gz   
S_ZIKA_23Mom  CKDN210020709-1A_HFW5VDSX3_L2  S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2_1.f...   
S_ZIKA_24Mom  CKDN210020710-1A_HFW5VDSX3_L3  S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3_1.f...   
S_ZIKA_25Baby CKDN210020712-1A_HFW5VDSX3_L2  S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2_1....   
S_ZIKA_25Mom  CKDN210020711-1A_HFW5VDSX3_L3  S_ZIKA_25Mom_CKDN210020711-1A_HFW5VDSX3_L3_1.f...   

                                                                      unit  \
sample_id     unit                                                           
RZ100         CKDN220028927-1A_H3FYWDSX5_L1  CKDN220028927-1A_H3FYWDSX5_L1   
              CKDN220028927-1A_H3G37DSX5_L3  CKDN220028927-1A_H3G37DSX5_L3   
RZ100M        CKDN22H000023-1A_HT3F3DSX5_L1  CKDN22H000023-1A_HT3F3DSX5_L1   
              CKDN22H000023-1A_HT3H7DSX5_L2  CKDN22H000023-1A_HT3H7DSX5_L2   
RZ101         CKDN220028929-1A_H3FYWDSX5_L1  CKDN220028929-1A_H3FYWDSX5_L1   
...                                                                    ...   
S_ZIKA_23B    CKDN210020708-1A_HFW5VDSX3_L1  CKDN210020708-1A_HFW5VDSX3_L1   
S_ZIKA_23Mom  CKDN210020709-1A_HFW5VDSX3_L2  CKDN210020709-1A_HFW5VDSX3_L2   
S_ZIKA_24Mom  CKDN210020710-1A_HFW5VDSX3_L3  CKDN210020710-1A_HFW5VDSX3_L3   
S_ZIKA_25Baby CKDN210020712-1A_HFW5VDSX3_L2  CKDN210020712-1A_HFW5VDSX3_L2   
S_ZIKA_25Mom  CKDN210020711-1A_HFW5VDSX3_L3  CKDN210020711-1A_HFW5VDSX3_L3   

                                                                             sample_name  \
sample_id     unit                                                                         
RZ100         CKDN220028927-1A_H3FYWDSX5_L1          RZ100_CKDN220028927-1A_H3FYWDSX5_L1   
              CKDN220028927-1A_H3G37DSX5_L3          RZ100_CKDN220028927-1A_H3G37DSX5_L3   
RZ100M        CKDN22H000023-1A_HT3F3DSX5_L1         RZ100M_CKDN22H000023-1A_HT3F3DSX5_L1   
              CKDN22H000023-1A_HT3H7DSX5_L2         RZ100M_CKDN22H000023-1A_HT3H7DSX5_L2   
RZ101         CKDN220028929-1A_H3FYWDSX5_L1          RZ101_CKDN220028929-1A_H3FYWDSX5_L1   
...                                                                                  ...   
S_ZIKA_23B    CKDN210020708-1A_HFW5VDSX3_L1     S_ZIKA_23B_CKDN210020708-1A_HFW5VDSX3_L1   
S_ZIKA_23Mom  CKDN210020709-1A_HFW5VDSX3_L2   S_ZIKA_23Mom_CKDN210020709-1A_HFW5VDSX3_L2   
S_ZIKA_24Mom  CKDN210020710-1A_HFW5VDSX3_L3   S_ZIKA_24Mom_CKDN210020710-1A_HFW5VDSX3_L3   
S_ZIKA_25Baby CKDN210020712-1A_HFW5VDSX3_L2  S_ZIKA_25Baby_CKDN210020712-1A_HFW5VDSX3_L2   
S_ZIKA_25Mom  CKDN210020711-1A_HFW5VDSX3_L3   S_ZIKA_25Mom_CKDN210020711-1A_HFW5VDSX3_L3   

                                                 sample_id  \
sample_id     unit                                           
RZ100         CKDN220028927-1A_H3FYWDSX5_L1          RZ100   
              CKDN220028927-1A_H3G37DSX5_L3          RZ100   
RZ100M        CKDN22H000023-1A_HT3F3DSX5_L1         RZ100M   
              CKDN22H000023-1A_HT3H7DSX5_L2         RZ100M   
RZ101         CKDN220028929-1A_H3FYWDSX5_L1          RZ101   
...                                                    ...   
S_ZIKA_23B    CKDN210020708-1A_HFW5VDSX3_L1     S_ZIKA_23B   
S_ZIKA_23Mom  CKDN210020709-1A_HFW5VDSX3

In [27]:
units.loc["RZ100"]


,file_name,unit,sample_name,sample_id,library_index,acc1,acc2,lane,R_pattern,R_sep,read_num,ext,tail,sample_number,matched_pattern,file2,PE
unit,,,,,,,,,,,,,,,,,
CKDN220028927-1A_H3FYWDSX5_L1,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_1.fq.gz,CKDN220028927-1A_H3FYWDSX5_L1,RZ100_CKDN220028927-1A_H3FYWDSX5_L1,RZ100,CKDN220028927-1A_H3FYWDSX5,CKDN220028927,1A_H3FYWDSX5,L1,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100_CKDN220028927-1A_H3FYWDSX5_L1_2.fq.gz,True
CKDN220028927-1A_H3G37DSX5_L3,RZ100_CKDN220028927-1A_H3G37DSX5_L3_1.fq.gz,CKDN220028927-1A_H3G37DSX5_L3,RZ100_CKDN220028927-1A_H3G37DSX5_L3,RZ100,CKDN220028927-1A_H3G37DSX5,CKDN220028927,1A_H3G37DSX5,L3,_1,_,1,fq.gz,NaN,NaN,Novagen1,RZ100_CKDN220028927-1A_H3G37DSX5_L3_2.fq.gz,True


In [40]:
units.loc[:, 'lane'][units['unit'] == 'CKDN220028927-1A_H3FYWDSX5_L1'].tolist()[0]

'L1'

In [23]:
(lambda X: [f" --INPUT 03_bamPrep/{X}_{b}.pqsr.bam" for b in units.loc[{X}, "unit"].tolist()])("RZ100")

[' --INPUT 03_bamPrep/RZ100_CKDN220028927-1A_H3FYWDSX5_L1.pqsr.bam',
 ' --INPUT 03_bamPrep/RZ100_CKDN220028927-1A_H3G37DSX5_L3.pqsr.bam']

NameError: name 'X' is not defined

In [ ]:
units.loc["RZ101M", "lane"].tolist()

['L1', 'L1']

In [ ]:
from snakemake.io import *


In [ ]:
# Define the input function for the trim rule
def trim_input_func(wildcards):
    sample_id, library_index, lane = wildcards.sample_id, wildcards.library_index, wildcards.lane
    pattern = "{sample_id}_{library_index}_{lane}_{read_num}.fq.gz"

    # Get the relevant rows from the samples DataFrame
    rows = unit.loc[sample_id, library_index, lane]

    # Generate the input files using the R_pattern and read_num columns
    input_files = expand(pattern, sample_id=rows['R_pattern'], read_num=rows['read_num'])

    return input_files


In [ ]:
PATH = "."

In [ ]:
def get_fastqs(sample):
    mydict = dict(
        zip(
            ["R1", "R2"],
                [
                    expand(
                            "{sample}_{unit}_{lane}_1.fq.gz",
                            unit=units.loc[sample, "library_index"].tolist(),
                            sample=sample,
                            lane=units.loc[sample, "lane"].unique().tolist()

                        ),
                    expand(
                            "{sample}_{unit}_{lane}_2.fq.gz",
                            unit=units.loc[sample, "library_index"].tolist(),
                            sample=sample,
                            lane=units.loc[sample, "lane"].unique().tolist()
                        )
                ]
        )
    )
    return mydict


In [ ]:
get_fastqs("RZ100")

{'R1': ['RZ100_CKDN220028927-1A_H3FYWDSX5_L1_1.fq.gz',
  'RZ100_CKDN220028927-1A_H3FYWDSX5_L3_1.fq.gz',
  'RZ100_CKDN220028927-1A_H3G37DSX5_L1_1.fq.gz',
  'RZ100_CKDN220028927-1A_H3G37DSX5_L3_1.fq.gz'],
 'R2': ['RZ100_CKDN220028927-1A_H3FYWDSX5_L1_2.fq.gz',
  'RZ100_CKDN220028927-1A_H3FYWDSX5_L3_2.fq.gz',
  'RZ100_CKDN220028927-1A_H3G37DSX5_L1_2.fq.gz',
  'RZ100_CKDN220028927-1A_H3G37DSX5_L3_2.fq.gz']}